# Week 15 Programming Assignment

The final thing for this semester that we haven't exercised, yet, is working with databases through Python.  In thi final assignment of the semester, you will practice pulling data from either Google Big Query or Snowflake, loading that data into a Pandas data frame, summarizing the data, and then exporting that to an Excel file.

**You will need to use your own Google Big Query or Snowflake account to run this notebook, but you should try to make it configurable so anyone with an account in those technologies could run your notebook with minimal changes.**


Build a notebook with good comments (either in code or in markdown cells).  Then submit your assignment as usual.

### 1. Connect to the data source

Follow the examples provided in the [week15 folder of our GitHub repository](https://github.com/paulboal/hds5210-2023/tree/main/week15) to connect your notebook either to Big Query or Snowflake.

In [1]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [3]:
%load_ext google.colab.data_table

The google.colab.data_table extension is already loaded. To reload it, use:
  %reload_ext google.colab.data_table


### 2. Query some data

Assuming the source database has some data in it or that you can load some data into it from any source, query it.  Then, read that data into a Pandas dataframe.

In [10]:
import pandas as pd
from google.cloud import bigquery

project_id = "crafty-key-407904"
client = bigquery.Client(project=project_id)

# Specify sample size
FIPS_Code= 4027

# Build a query that gets sample ids then joins to get data
query = """
   WITH samples AS (
    SELECT RAND() AS rnd, *
    FROM `crafty-key-407904.hds_assignment.unemployment`
),
sample_ids AS (
    SELECT FIPS_Code
    FROM `crafty-key-407904.hds_assignment.unemployment`
    ORDER BY State
)
SELECT *
FROM samples
JOIN sample_ids
ON samples.FIPS_Code = sample_ids.FIPS_Code;

"""
# Parameterize query and run it
job_config = bigquery.QueryJobConfig(
    query_parameters=[
        bigquery.ScalarQueryParameter("FIPS_Code", "INT64",FIPS_Code)
    ]
)

State= client.query(query, job_config=job_config).to_dataframe()

print(State.shape)

(3277, 102)


### 3. Aggregate your data frame

Do some kind of aggregation on your data frame.  Something that makes sense and has some groups to it.  Don't just sum up one column for the entire data frame.  Be more creative than that.

In [13]:
State= pd.DataFrame({
    'FIPS_Code': [2063, 2066, 2201, 2232, 2280, 72000, 72007],
    'State': ["AK", "PR", "AL", "AZ", "CA", "Unknown1", "Unknown2"]
})
State_grouped = State.groupby('FIPS_Code').agg({'State': 'sum'}).reset_index()

### 4. Write to Excel

Use Pandas functions to write your summarized data out to a local Excel file.

In [14]:
!pip install xlsxwriter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 2.3 MB/s eta 0:00:00


In [16]:
import xlsxwriter
writer = pd.ExcelWriter('output.xlsx', engine='xlsxwriter')
State_grouped.to_excel(writer, index=False, sheet_name='State')
writer.save()

<ipython-input-16-bdb5cbbb2257>:4: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()


---

## Submitting Your Work

Submit your work as usual